In [14]:
import pandas as pd
import json
import os
import numpy as np
import shutil


In [16]:
instance_path = "instances_IB"
instancs_list_file = "instances_bilevel_IB.csv" #os.path.join("instances_IB", "instances_IB.csv")
df = pd.read_csv(instancs_list_file, dtype={'himprov': float})
df

,id,seed,DeltaRandom,DDecrease,D_t,B,LT,LS,LF,LL,delta,lobjval,lz1,lz2,fobjval,tot_runtime,gapf,best_it,himprov,solved
0,IB_B1015_D1000_F3_L4_T_5_seed_0,0,False,False,1000,1015,5,3,3,4,0.7,87989.66,86977.00,1012.66,54401.00,1.77,0.0,26,-inf,True
1,IB_B1181_D1000_F3_L4_T_5_seed_1,1,False,False,1000,1181,5,3,3,4,0.7,139147.65,137985.00,1162.65,59181.00,1.94,0.0,3,-inf,True
2,IB_B1137_D1000_F3_L4_T_5_seed_2,2,False,False,1000,1137,5,3,3,4,0.7,32798.98,29798.98,3000.00,65095.81,1.56,0.0,5,0.0,False
3,IB_B1099_D1000_F3_L4_T_5_seed_3,3,False,False,1000,1099,5,3,3,4,0.7,60181.05,57181.05,3000.00,62185.55,1.37,0.0,1,0.0,False
4,IB_B1189_D1000_F3_L4_T_5_seed_4,4,False,False,1000,1189,5,3,3,4,0.7,66708.18,63708.18,3000.00,58563.86,1.23,0.0,1,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,IB_B1491_D833_F12_L12_T_15_seed_0,0,False,False,833,1491,15,12,12,12,0.7,694484.29,684992.00,9492.29,121421.00,2.67,0.0,1,0.0,False
716,IB_B1547_D833_F12_L12_T_15_seed_1,1,False,False,833,1547,15,12,12,12,0.7,617535.04,608269.44,9265.60,123991.00,2.52,0.0,1,0.0,False
717,IB_B1491_D833_F12_L12_T_15_seed_2,2,False,False,833,1491,15,12,12,12,0.7,610514.11,600933.00,9581.11,122955.00,2.37,0.0,3,0.0,False
718,IB_B1490_D833_F12_L12_T_15_seed_3,3,False,False,833,1490,15,12,12,12,0.7,945522.90,936089.00,9433.90,119524.00,5.29,0.0,4,0.0,False


In [13]:
df.columns

Index(['id', 'seed', 'DeltaRandom', 'DDecrease', 'D_t', 'B', 'LT', 'LS', 'LF',
       'LL', 'delta', 'lobjval', 'lz1', 'lz2', 'fobjval', 'tot_runtime',
       'gapf', 'best_it', 'himprov', 'solved'],
      dtype='object')

In [17]:
for index, row in df.iterrows():
    instance_name = row['id']
    print(f"Processing instance number {index+1}, name: {instance_name}")
    instance_file = os.path.join(instance_path, f"{instance_name}.xlsx")
    dfkappa = pd.read_excel(instance_file, sheet_name='kappa')
    dfGeneral = pd.read_excel(instance_file, sheet_name='General', index_col='parameter')
    B = int(dfGeneral.loc['B', 'value'])
    D = row['D_t']
    LF = int(dfGeneral.loc['LF', 'value'])
    LT = int(dfGeneral.loc['LT', 'value'])
    LL = int(dfGeneral.loc['LL', 'value'])
    seed = row['seed']
    #print(B)
    updated_B = int(LF*dfkappa['kappa'].mean()*1.5) #=INT(AVERAGE($kappa.C:C)*B8)*1.3#
    #print(updated_B)
    dfGeneral.at['B', 'value'] = updated_B
    df.at[index, 'B'] = updated_B
    df.at[index,'lobjval'] = -1.0
    df.at[index,'lz1'] = -1.0
    df.at[index,'lz2'] = -1.0
    df.at[index,'fobjval'] = -1.0
    df.at[index,'tot_runtime'] = -1.0
    df.at[index,'gapf'] = -1.0
    df.at[index,'best_it'] = -1.0
    df.at[index,'himprov'] = -1.0
    df.at[index, 'solved'] = False
    updated_instance_name = f'IB_B{updated_B}_D{D}_F{LF}_L{LL}_T_{LT}_seed_{seed}'
    print(f"Updated instance name: {updated_instance_name}")
    df.at[index, 'id'] = updated_instance_name
    with pd.ExcelWriter(instance_file, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        dfGeneral.loc['B', 'value'] = updated_B
        dfGeneral.to_excel(writer, sheet_name='General', index=True)
    #try:
    #    shutil.copy(instance_file, os.path.join(instance_path, f"{updated_instance_name}.xlsx"))
    #except Exception as e:
    #    print(f"Error copying file for instance {instance_name}: {e}")
    os.rename(instance_file, os.path.join(instance_path, f"{updated_instance_name}.xlsx"))
df.to_csv(instancs_list_file, index=False)

Processing instance number 1, name: IB_B1015_D1000_F3_L4_T_5_seed_0
Updated instance name: IB_B1171_D1000_F3_L4_T_5_seed_0
Processing instance number 2, name: IB_B1181_D1000_F3_L4_T_5_seed_1
Updated instance name: IB_B1363_D1000_F3_L4_T_5_seed_1
Processing instance number 3, name: IB_B1137_D1000_F3_L4_T_5_seed_2
Updated instance name: IB_B1312_D1000_F3_L4_T_5_seed_2
Processing instance number 4, name: IB_B1099_D1000_F3_L4_T_5_seed_3
Updated instance name: IB_B1268_D1000_F3_L4_T_5_seed_3
Processing instance number 5, name: IB_B1189_D1000_F3_L4_T_5_seed_4
Updated instance name: IB_B1372_D1000_F3_L4_T_5_seed_4
Processing instance number 6, name: IB_B1692_D1666_F3_L4_T_5_seed_0
Updated instance name: IB_B1952_D1666_F3_L4_T_5_seed_0
Processing instance number 7, name: IB_B1969_D1666_F3_L4_T_5_seed_1
Updated instance name: IB_B2272_D1666_F3_L4_T_5_seed_1
Processing instance number 8, name: IB_B1896_D1666_F3_L4_T_5_seed_2
Updated instance name: IB_B2188_D1666_F3_L4_T_5_seed_2
Processing insta